In [ ]:
import cloudscraper 
from bs4 import BeautifulSoup
import json
import pandas as pd
import sqlite3
import os 
import datetime as dt
import requests

## Renda Fixa

In [ ]:
# Coletar URl para Renda Fixa
'''url = 'https://investidor10.com.br/renda-fixa/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')'''
# O site tem cloudflare, portanto vamos tentar com o Agent
'''
url = 'https://investidor10.com.br/renda-fixa/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')'''

### Utilizar a API do site

In [ ]:
class Scraper:
    def __init__(self):

        
    def pagination(self):
    
        while True:
            api_url = f"https://investidor10.com.br/api/fixed-incomes?page={self.page}&filter=&order="
        
            try:
                response = self.scraper.get(api_url)
                response.raise_for_status()
                
                data = response.json()
                
                # A API retorna um dicionário com a chave 'data', que contém a lista de itens.
                # Se 'data' estiver vazia, significa que chegamos ao fim da paginação.
                page_data = data.get('data', [])
                if not page_data:
                    print(f"Página {self.page} não tem mais dados. Paginação concluída.")
                    break # Sai do loop
                    
                self.all_data.extend(page_data) # Adiciona os dados da página atual à lista principal
                print(f"Página {self.page} coletada. {len(page_data)} itens encontrados.")
                
                self.page += 1 # Prepara para a próxima página
                
            except Exception as e:
                print(f"Ocorreu um erro ao coletar a página {self.page}: {e}")
                break # Sai do loop em caso de erro

        # Agora você tem todos os dados na variável `self.all_data`
        if self.all_data:
            print(f"\n Coletados {len(self.all_data)} títulos de renda fixa no total.")
            # Você pode inspecionar o primeiro item para ver a estrutura dos dados
            print("\n Exemplo do primeiro item coletado:")
            print(json.dumps(self.all_data[0], indent=4))
        else:
            print("Nenhum dado de renda fixa foi coletado.")
        
        
        # Colocar os Dados dentro do Dataframe
        self.df = pd.DataFrame(self.all_data)       
        
        return self.df
    
    
            
scraper = Scraper()
data = scraper.pagination()


Página 1 coletada. 30 itens encontrados.
Página 2 coletada. 30 itens encontrados.
Página 3 coletada. 30 itens encontrados.
Página 4 coletada. 30 itens encontrados.
Página 5 coletada. 30 itens encontrados.
Página 6 coletada. 30 itens encontrados.
Página 7 coletada. 30 itens encontrados.
Página 8 coletada. 30 itens encontrados.
Página 9 coletada. 30 itens encontrados.
Página 10 coletada. 30 itens encontrados.
Página 11 coletada. 30 itens encontrados.
Página 12 coletada. 30 itens encontrados.
Página 13 coletada. 11 itens encontrados.
Página 14 não tem mais dados. Paginação concluída.

 Coletados 371 títulos de renda fixa no total.

 Exemplo do primeiro item coletado:
{
    "id": 5665,
    "name": "CRI Dasa IPCA + 10,20%",
    "slug": "cri-dasa-ipca-1020-5665",
    "type": "CRI",
    "fgc_guarantee": 0,
    "active": 1,
    "minimum_investment": 993.73,
    "financial_risk": "Baixo",
    "daily_liquidity": "No Vencimento",
    "taxation": "EXEMPT_INCOME",
    "indexer_profitability": "IPCA

In [ ]:
class Etl:
    '''
    Classe para realizar operações de ETL (Extract, Transform, Load) em dados financeiros.
    
    Métodos:
    - fillna_columns: Preenche valores NaN em colunas específicas com zero.
    - transform: Aplica transformações nos dados, incluindo o preenchimento de NaNs.
    - load: Carrega os dados transformados em um banco de dados SQLite.
    - extract: Extrai dados de uma fonte (a ser implementado).
    - __init__: Inicializa a classe com um scraper, uma lista para armazenar dados e um DataFrame vazio.
    
    Atributos:
    - scraper: Objeto para realizar requisições web.
    - all_data: Lista para armazenar todos os dados coletados.
    - page: Número da página atual para paginação.
    - df: DataFrame do pandas para manipulação de dados.
    
    '''
    def __init__(self):
        self.scraper = cloudscraper.create_scraper()
        self.all_data = []
        self.page = 1
        self.df = pd.DataFrame()
    
    
    def fillna_columns(self, df: pd.DataFrame, columns: list):
        """
        Preenche valores NaN em colunas específicas com zero.
        Se a coluna não existir, lança uma exceção personalizada.
        """
        for column in columns:
            try:
                if column in df.columns:
                    df[column].fillna(0, inplace=True)
                else:
                    raise KeyError(f"A coluna '{column}' não existe no DataFrame.")
            except Exception as e:
                print(f"Erro ao processar a coluna '{column}': {e}")
        
        return df


    def extract(self):
        
        while True:
            api_url = f"https://investidor10.com.br/api/fixed-incomes?page={self.page}&filter=&order="
        
            try:
                response = self.scraper.get(api_url)
                response.raise_for_status()
                
                data = response.json()
                
                # A API retorna um dicionário com a chave 'data', que contém a lista de itens.
                # Se 'data' estiver vazia, significa que chegamos ao fim da paginação.
                page_data = data.get('data', [])
                if not page_data:
                    print(f"Página {self.page} não tem mais dados. Paginação concluída.")
                    break # Sai do loop
                    
                self.all_data.extend(page_data) # Adiciona os dados da página atual à lista principal
                print(f"Página {self.page} coletada. {len(page_data)} itens encontrados.")
                
                self.page += 1 # Prepara para a próxima página
                
            except Exception as e:
                print(f"Ocorreu um erro ao coletar a página {self.page}: {e}")
                break # Sai do loop em caso de erro

        # Agora você tem todos os dados na variável `self.all_data`
        if self.all_data:
            print(f"\n Coletados {len(self.all_data)} títulos de renda fixa no total.")
            # Você pode inspecionar o primeiro item para ver a estrutura dos dados
            print("\n Exemplo do primeiro item coletado:")
            print(json.dumps(self.all_data[0], indent=4))
        else:
            print("Nenhum dado de renda fixa foi coletado.")
        
        
        # Colocar os Dados dentro do Dataframe
        self.df = pd.DataFrame(self.all_data)       
        
        return self.df
    
    def transform(self, data):
        columns_to_fillna = ['company_id','stock_bdr_id','slug','thumbnail'] # Colunas a preencher com zero
        self.fillna_columns(data, columns_to_fillna).info()
        
        return data

In [ ]:


class Db:
    '''
    Classe para gerenciar a conexão com o banco de dados SQLite e operações básicas.
    '''
    def __init__(self):
        self.db_path = os.path.join(os.getcwd(), '..', '..', 'datawarehouse')
        self.db_file = os.path.join(self.db_path, 'datawarehouse.sqlite3')
        self.conn = sqlite3.connect(self.db_file) 
        self.cursor = self.conn.cursor()
        print(f"Conectado ao banco de dados SQLite")
    
    def drop_table(self, table_name: str):
        '''
        Remove uma tabela do banco de dados se ela existir.
        '''
        try:
            self.cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
            self.conn.commit()
            print(f"Tabela '{table_name}' removida com sucesso.")
        
        except Exception as e:
            print(f"Erro ao remover a tabela '{table_name}': {e}")
    
    def to_sql(self, df: pd.DataFrame, table_name: str, if_exists_: str):
        '''
        Carrega um DataFrame em uma tabela do banco de dados. Se a tabela já existir, ela será substituída.
        '''
        try:
            df['landing_date'] = dt.date.today()  # Adiciona a coluna 'landing_date' com a data e hora atual
            df.to_sql(table_name, self.conn, if_exists=if_exists_, index=False)
            print(f"Dados carregados na tabela '{table_name}' com sucesso.")
        
        except Exception as e:
            print(f"Erro ao carregar dados na tabela '{table_name}': {e}")
            
        finally:
            if self.conn:
                self.conn.close()
                print("Conexão com o banco de dados fechada.")
    
database = Db()
database.drop_table('renda_fixa')
#database.to_sql(data, 'renda_fixa', if_exists_='append')

Conectado ao banco de dados SQLite
Tabela 'renda_fixa' removida com sucesso.


# Ações

In [213]:
class EtlAcoes:
    """
    ETL para coletar dados de ações do site Investidor10.
    """

    def __init__(self):
        self.base_url = "https://investidor10.com.br/acoes/rankings/maiores-valor-de-mercado/"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) "
                          "Chrome/120.0.0.0 Safari/537.36"
        }
        self.page = 1
        self.all_data = []
        self.df = pd.DataFrame()

    def extract(self):
        """
        Extrai todas as páginas até não encontrar mais linhas de ações.
        """
        while True:
            url = f"{self.base_url}?page={self.page}"
            print(f"Coletando página {self.page}...")

            response = requests.get(url, headers=self.headers)
            if response.status_code != 200:
                print(f"Erro ao acessar página {self.page}: {response.status_code}")
                break

            soup = BeautifulSoup(response.content, "html.parser")
            rows = soup.find_all("tr")

            page_data = []
            for row in rows:
                info = {}
                ticker_span = row.find("span", class_="font-semibold text-[#14171F] group-hover:text-[#485063]")
                if not ticker_span:
                    continue
                info["ticker"] = ticker_span.get_text(strip=True)

                for td in row.find_all("td", class_="sorting"):
                    nome = td.get("data-name")
                    if nome:
                        valor = td.get_text(strip=True)
                        info[nome] = valor

                page_data.append(info)

            if not page_data:
                print(f"Página {self.page} não contém mais dados. Paginação concluída.")
                break

            self.all_data.extend(page_data)
            self.page += 1  # próxima página

        self.df = pd.DataFrame(self.all_data)
        print(f"Total de registros coletados: {len(self.df)}")
        return self.df

In [312]:
acoes_inst = EtlAcoes()
acoes = acoes_inst.extract()

Coletando página 1...
Coletando página 2...
Coletando página 3...
Coletando página 4...
Coletando página 5...
Coletando página 6...
Página 6 não contém mais dados. Paginação concluída.
Total de registros coletados: 240


In [313]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years,variation_30_days,variation_12_months,current_price,bazin_price,...,balance_net_revenue,growth_net_revenue_last_5_years,growth_net_profit_last_5_years,balance_availability,gross_debt_net_worth,dividend_yield_last_12_months,dividend_yield_last_5_years,name_sector,name_subsector,name_segment
0,PETR4,"421,25 B",90,"5,22","1,01","430,10%","0,80%","-1,28%","R$ 31,35","R$ 131,90",...,"493,12 B","12,63%","65,59%","51,85 B","0,93","16,27%","25,73%","Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,"396,98 B",100,"9,82","2,02","195,94%","1,40%","27,05%","R$ 39,03","R$ 27,37",...,"369,42 B","16,28%","23,81%","32,18 B",-,"7,24%","4,82%",Financeiro,Intermediários Financeiros,Bancos
2,VALE3,"261,36 B",70,"9,03","1,22","53,84%","3,64%","-5,07%","R$ 57,58","R$ 125,67",...,"209,60 B","0,10%","2,28%","31,09 B","0,45","7,95%","9,55%",Materiais Básicos,Mineração,Minerais Metálicos
3,BPAC11,"240,70 B",100,"13,82","2,91","192,68%","7,69%","47,18%","R$ 48,31","R$ 11,96",...,"50,13 B","30,59%","32,53%","1,38 B",-,"2,16%","2,23%",Financeiro,Intermediários Financeiros,Bancos
4,ABEV3,"190,24 B",70,"12,90","2,06","18,95%","-2,27%","-1,53%","R$ 12,07","R$ 12,13",...,"91,72 B","9,46%","5,30%","17,52 B","0,03","8,48%","4,66%",Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,CAMB3,"394,00 M",80,"5,66","1,28","145,74%","-1,06%","-20,15%","R$ 9,32","R$ 7,25",...,"412,43 M","21,39%","25,88%","60,38 M","0,01","10,00%","3,80%",Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
236,MEAL3,"378,01 M",40,"-2,87","0,40","-59,13%",-,"-1,49%","R$ 1,32",R$ -,...,"2,11 B","12,88%",-,"297,72 M","0,58",-,-,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares
237,RNEW4,"374,66 M",30,"-2,91","0,28","-82,51%","-13,91%","-50,99%","R$ 0,99",R$ -,...,"394,64 M","41,12%",-,"87,40 M","0,57",-,-,Utilidade Pública,Energia Elétrica,Energia Elétrica
238,AVLL3,"320,34 M",20,"-5,35","-2,38",-,"-5,20%","-59,31%","R$ 1,64",R$ -,...,"685,95 M","30,65%",-,"210,25 M","-5,24",-,-,Consumo Cíclico,Construção Civil,Incorporações


In [314]:
acoes.columns

Index(['ticker', 'enterprise_value', 'rate', 'p_l', 'p_vp',
       'variation_5_years', 'variation_30_days', 'variation_12_months',
       'current_price', 'bazin_price', 'bazin_upside', 'graham_price',
       'graham_upside', 'roe', 'net_margin', 'balance_net_profit',
       'balance_net_revenue', 'growth_net_revenue_last_5_years',
       'growth_net_profit_last_5_years', 'balance_availability',
       'gross_debt_net_worth', 'dividend_yield_last_12_months',
       'dividend_yield_last_5_years', 'name_sector', 'name_subsector',
       'name_segment'],
      dtype='object')

In [315]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years,variation_30_days,variation_12_months,current_price,bazin_price,...,balance_net_revenue,growth_net_revenue_last_5_years,growth_net_profit_last_5_years,balance_availability,gross_debt_net_worth,dividend_yield_last_12_months,dividend_yield_last_5_years,name_sector,name_subsector,name_segment
0,PETR4,"421,25 B",90,"5,22","1,01","430,10%","0,80%","-1,28%","R$ 31,35","R$ 131,90",...,"493,12 B","12,63%","65,59%","51,85 B","0,93","16,27%","25,73%","Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,"396,98 B",100,"9,82","2,02","195,94%","1,40%","27,05%","R$ 39,03","R$ 27,37",...,"369,42 B","16,28%","23,81%","32,18 B",-,"7,24%","4,82%",Financeiro,Intermediários Financeiros,Bancos
2,VALE3,"261,36 B",70,"9,03","1,22","53,84%","3,64%","-5,07%","R$ 57,58","R$ 125,67",...,"209,60 B","0,10%","2,28%","31,09 B","0,45","7,95%","9,55%",Materiais Básicos,Mineração,Minerais Metálicos
3,BPAC11,"240,70 B",100,"13,82","2,91","192,68%","7,69%","47,18%","R$ 48,31","R$ 11,96",...,"50,13 B","30,59%","32,53%","1,38 B",-,"2,16%","2,23%",Financeiro,Intermediários Financeiros,Bancos
4,ABEV3,"190,24 B",70,"12,90","2,06","18,95%","-2,27%","-1,53%","R$ 12,07","R$ 12,13",...,"91,72 B","9,46%","5,30%","17,52 B","0,03","8,48%","4,66%",Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,CAMB3,"394,00 M",80,"5,66","1,28","145,74%","-1,06%","-20,15%","R$ 9,32","R$ 7,25",...,"412,43 M","21,39%","25,88%","60,38 M","0,01","10,00%","3,80%",Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
236,MEAL3,"378,01 M",40,"-2,87","0,40","-59,13%",-,"-1,49%","R$ 1,32",R$ -,...,"2,11 B","12,88%",-,"297,72 M","0,58",-,-,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares
237,RNEW4,"374,66 M",30,"-2,91","0,28","-82,51%","-13,91%","-50,99%","R$ 0,99",R$ -,...,"394,64 M","41,12%",-,"87,40 M","0,57",-,-,Utilidade Pública,Energia Elétrica,Energia Elétrica
238,AVLL3,"320,34 M",20,"-5,35","-2,38",-,"-5,20%","-59,31%","R$ 1,64",R$ -,...,"685,95 M","30,65%",-,"210,25 M","-5,24",-,-,Consumo Cíclico,Construção Civil,Incorporações


In [316]:
def split_value_and_scale(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Separa uma coluna com valores tipo '427,98 B' em duas:
      - <coluna>_num   -> valor numérico (float)
      - <coluna>_scale -> escala (M, B ou None)
    """
    df = df.copy()  # evita mexer no DF original

    # Garante que a coluna é string
    df[column] = df[column].astype(str)

    # Extrai números
    df[f"{column}_num"] = (
        df[column]
        .str.extract(r"([\d\.,]+)")[0]  # pega só o grupo
        .replace(r"\.(?=\d{3}(?:\.|,))", "", regex=True)  # remove milhar
        .str.replace(",", ".", regex=True)  # vírgula decimal -> ponto
        .astype(float)
    )

    # Extrai escalas (M ou B)
    df[f"{column}_scale"] = df[column].str.extract(r"([MB])")

    return df

def convert_br_number(series: pd.Series) -> pd.Series:
    """
    Converte uma Series com números no formato brasileiro (ex: '1.423,42')
    para float no formato Python (ex: 1423.42).
    """
    return (
        series.astype(str)
        .str.replace('%','')
        .str.replace('R$ ','')
        .str.replace('.', '')   # remove separador de milhar
        .str.replace(',', '.') # troca vírgula decimal por ponto
        .str.replace('-','0') # troca o '-' por '0' para a conversão para float
        .astype(float)
    )



In [317]:
money_columns = ["enterprise_value", "balance_net_profit", "balance_net_revenue"]

In [318]:
# Colunas para limpeza
# Colunas em porcentagem
columns_with_percentage = [
    'variation_5_years','variation_30_days',
    'variation_12_months','bazin_upside', 
    'graham_upside', 'roe', 'net_margin',
    'growth_net_revenue_last_5_years',
    'growth_net_profit_last_5_years',
    'dividend_yield_last_12_months',
    'dividend_yield_last_5_years'
]

columns_with_real_sign = [
    'graham_price',
    'bazin_price',
    'current_price',
]

columns_with_B_sign = [
    'enterprise_value',
    'balance_net_profit',
    'balance_net_revenue',
    'balance_availability'    
]

columns_numeric_with_comma = [
    'p_l','p_vp','current_price',
    'bazin_price','graham_price',
    'gross_debt_net_worth',
    'current_price'
]

In [319]:
for money_col in money_columns:
    acoes = split_value_and_scale(acoes, money_col)

In [320]:
for comma in columns_numeric_with_comma:
    acoes[comma] = convert_br_number(acoes[comma])

In [321]:
for perce in columns_with_percentage:
    acoes[perce] = convert_br_number(acoes[perce])
    acoes.rename(
        columns={perce:f'{perce}_percentage'},inplace=True
    )

In [324]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years_percentage,variation_30_days_percentage,variation_12_months_percentage,current_price,bazin_price,...,dividend_yield_last_5_years_percentage,name_sector,name_subsector,name_segment,enterprise_value_num,enterprise_value_scale,balance_net_profit_num,balance_net_profit_scale,balance_net_revenue_num,balance_net_revenue_scale
0,PETR4,"421,25 B",90,5.22,1.01,430.10,0.80,1.28,3135.0,131.90,...,25.73,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",421.25,B,77.82,B,493.12,B
1,ITUB4,"396,98 B",100,9.82,2.02,195.94,1.40,27.05,3903.0,27.37,...,4.82,Financeiro,Intermediários Financeiros,Bancos,396.98,B,43.82,B,369.42,B
2,VALE3,"261,36 B",70,9.03,1.22,53.84,3.64,5.07,5758.0,125.67,...,9.55,Materiais Básicos,Mineração,Minerais Metálicos,261.36,B,27.87,B,209.60,B
3,BPAC11,"240,70 B",100,13.82,2.91,192.68,7.69,47.18,4831.0,11.96,...,2.23,Financeiro,Intermediários Financeiros,Bancos,240.70,B,13.41,B,50.13,B
4,ABEV3,"190,24 B",70,12.90,2.06,18.95,2.27,1.53,1207.0,12.13,...,4.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,190.24,B,15.19,B,91.72,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,CAMB3,"394,00 M",80,5.66,1.28,145.74,1.06,20.15,932.0,7.25,...,3.80,Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados,394.00,M,69.62,M,412.43,M
236,MEAL3,"378,01 M",40,2.87,0.40,59.13,0.00,1.49,132.0,0.00,...,0.00,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares,378.01,M,131.67,M,2.11,B
237,RNEW4,"374,66 M",30,2.91,0.28,82.51,13.91,50.99,99.0,0.00,...,0.00,Utilidade Pública,Energia Elétrica,Energia Elétrica,374.66,M,126.76,M,394.64,M
238,AVLL3,"320,34 M",20,5.35,2.38,0.00,5.20,59.31,164.0,0.00,...,0.00,Consumo Cíclico,Construção Civil,Incorporações,320.34,M,37.60,M,685.95,M


In [328]:
acoes[['ticker', 'enterprise_value_num','enterprise_value_scale','rate', 'p_l', 'p_vp',
       'variation_5_years_percentage', 'variation_30_days_percentage', 'variation_12_months_percentage',
       'current_price', 'bazin_price', 'bazin_upside_percentage', 'graham_price',
       'graham_upside_percentage', 'roe_percentage', 'net_margin_percentage', 'balance_net_profit_num','balance_net_profit_scale',
       'balance_net_revenue_num','balance_net_revenue_scale', 'growth_net_revenue_last_5_years_percentage',
       'growth_net_profit_last_5_years_percentage', 'balance_availability',
       'gross_debt_net_worth', 'dividend_yield_last_12_months_percentage',
       'dividend_yield_last_5_years_percentage', 'name_sector', 'name_subsector',
       'name_segment']]

,ticker,enterprise_value_num,enterprise_value_scale,rate,p_l,p_vp,variation_5_years_percentage,variation_30_days_percentage,variation_12_months_percentage,current_price,...,balance_net_revenue_scale,growth_net_revenue_last_5_years_percentage,growth_net_profit_last_5_years_percentage,balance_availability,gross_debt_net_worth,dividend_yield_last_12_months_percentage,dividend_yield_last_5_years_percentage,name_sector,name_subsector,name_segment
0,PETR4,421.25,B,90,5.22,1.01,430.10,0.80,1.28,3135.0,...,B,12.63,65.59,"51,85 B",0.93,16.27,25.73,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,396.98,B,100,9.82,2.02,195.94,1.40,27.05,3903.0,...,B,16.28,23.81,"32,18 B",0.00,7.24,4.82,Financeiro,Intermediários Financeiros,Bancos
2,VALE3,261.36,B,70,9.03,1.22,53.84,3.64,5.07,5758.0,...,B,0.10,2.28,"31,09 B",0.45,7.95,9.55,Materiais Básicos,Mineração,Minerais Metálicos
3,BPAC11,240.70,B,100,13.82,2.91,192.68,7.69,47.18,4831.0,...,B,30.59,32.53,"1,38 B",0.00,2.16,2.23,Financeiro,Intermediários Financeiros,Bancos
4,ABEV3,190.24,B,70,12.90,2.06,18.95,2.27,1.53,1207.0,...,B,9.46,5.30,"17,52 B",0.03,8.48,4.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,CAMB3,394.00,M,80,5.66,1.28,145.74,1.06,20.15,932.0,...,M,21.39,25.88,"60,38 M",0.01,10.00,3.80,Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
236,MEAL3,378.01,M,40,2.87,0.40,59.13,0.00,1.49,132.0,...,B,12.88,0.00,"297,72 M",0.58,0.00,0.00,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares
237,RNEW4,374.66,M,30,2.91,0.28,82.51,13.91,50.99,99.0,...,M,41.12,0.00,"87,40 M",0.57,0.00,0.00,Utilidade Pública,Energia Elétrica,Energia Elétrica
238,AVLL3,320.34,M,20,5.35,2.38,0.00,5.20,59.31,164.0,...,M,30.65,0.00,"210,25 M",5.24,0.00,0.00,Consumo Cíclico,Construção Civil,Incorporações


TypeError: can't multiply sequence by non-int of type 'float'

In [170]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years,variation_30_days,variation_12_months,current_price,bazin_price,...,balance_net_revenue,growth_net_revenue_last_5_years,growth_net_profit_last_5_years,balance_availability,gross_debt_net_worth,dividend_yield_last_12_months,dividend_yield_last_5_years,name_sector,name_subsector,name_segment
0,PETR4,4.279800e+11,90,"5,30","1,03","446,41%","2,32%","-0,07%","R$ 31,82","R$ 131,90",...,4.931200e+11,"12,63%","65,59%","51,85 B","0,93","16,03%","25,73%","Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,3.951900e+11,100,"9,80","2,01","196,62%","1,12%","24,51%","R$ 38,92","R$ 27,37",...,3.694200e+11,"16,28%","23,81%","32,18 B",-,"7,25%","4,82%",Financeiro,Intermediários Financeiros,Bancos
2,VALE3,2.607700e+11,70,"9,01","1,22","55,49%","3,40%","-5,95%","R$ 57,45","R$ 133,69",...,2.096000e+11,"0,10%","2,28%","31,09 B","0,45","7,98%","9,55%",Materiais Básicos,Mineração,Minerais Metálicos
3,BPAC11,2.434100e+11,100,"13,94","2,94","202,62%","8,67%","44,91%","R$ 48,75","R$ 11,96",...,5.013000e+10,"30,59%","32,53%","1,38 B",-,"2,20%","2,23%",Financeiro,Intermediários Financeiros,Bancos
4,ABEV3,1.959200e+11,70,"13,28","2,12","24,38%","0,65%","2,66%","R$ 12,43","R$ 12,13",...,9.172000e+10,"9,46%","5,30%","17,52 B","0,03","8,44%","4,66%",Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
5,BBDC3,1.754400e+11,90,"7,83","0,93","39,86%","6,09%","25,04%","R$ 15,30","R$ 17,58",...,2.344100e+11,"24,03%","4,85%","222,83 B",-,"9,29%","6,18%",Financeiro,Intermediários Financeiros,Bancos
6,WEGE3,1.521900e+11,100,"23,74","6,85","19,23%","-3,72%","-34,29%","R$ 36,26","R$ 10,35",...,4.097000e+10,"18,58%","23,05%","4,98 B","0,12","2,31%","1,25%",Bens Industriais,Máquinas e Equipamentos,"Motores, Compressores e Outros"
7,ELET3,1.439600e+11,50,"20,49","1,14","111,83%","16,48%","44,76%","R$ 52,44","R$ 24,37",...,4.368000e+10,"8,48%","0,75%","29,84 B","0,59","6,94%","3,54%",Utilidade Pública,Energia Elétrica,Energia Elétrica
8,BBAS3,1.264800e+11,70,"7,98","0,69","112,78%","3,18%","-15,53%","R$ 22,07","R$ 31,09",...,2.895100e+11,"18,92%","7,14%","64,94 B",-,"7,74%","8,50%",Financeiro,Intermediários Financeiros,Bancos
9,ITSA4,1.264600e+11,90,"8,15","1,41","153,69%","2,58%","17,53%","R$ 11,51","R$ 11,21",...,8.330000e+09,"7,21%","16,35%","7,32 B","0,13","8,98%","6,28%",Financeiro,Holdings Diversificadas,Holdings Diversificadas
